In [12]:
import pandas as pd
import numpy as np
import re
# import nltk
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
# from nltk import PorterStemmer
# nltk.download("wordnet")
# from nltk import WordNetLemmatizer
import spacy
stopwords = set(STOPWORDS) 

In [11]:
data = pd.read_excel("Email_Classificaion.xlsx")
data.head()

,Tower Name,Application Name,Email Subject,Flag
0,Core Pharmacy,PharmRDS-Accredo RealTime Messages,Production - Accredo Real Time Order Process S...,Actionable
1,Core Pharmacy,IVR - Web Service,Splunk Alert: SO IVR - Pharmacy Migration Coor...,Actionable
2,Core Pharmacy,ISAM- Web Service,Splunk Alert: ISAM - Slowness,Actionable
3,Core Pharmacy,Migration Co-Ordinaor,Splunk Alert: Migration_Coordinator_500 Clone,Actionable
4,Core Pharmacy,Claim Realm - Web Service,Splunk Alert: Claim Realm Service - Slowness A...,Actionable


In [13]:
data.isnull().sum()

Tower Name          0
Application Name    0
Email Subject       0
Flag                7
dtype: int64

In [14]:
# ger rid of na values in Flag
data = data[~data['Flag'].isna()]
print(data.shape)
data.head()

(66, 4)


,Tower Name,Application Name,Email Subject,Flag
0,Core Pharmacy,PharmRDS-Accredo RealTime Messages,Production - Accredo Real Time Order Process S...,Actionable
1,Core Pharmacy,IVR - Web Service,Splunk Alert: SO IVR - Pharmacy Migration Coor...,Actionable
2,Core Pharmacy,ISAM- Web Service,Splunk Alert: ISAM - Slowness,Actionable
3,Core Pharmacy,Migration Co-Ordinaor,Splunk Alert: Migration_Coordinator_500 Clone,Actionable
4,Core Pharmacy,Claim Realm - Web Service,Splunk Alert: Claim Realm Service - Slowness A...,Actionable


In [15]:
data['Flag'].value_counts()

Actionable        48
Non Actionable    18
Name: Flag, dtype: int64

In [16]:
data_copy = data.copy()

In [17]:
data_copy['combined'] = data_copy.apply(lambda x : f"{x[0]} {x[1]} {x[2]}",axis = 1)

In [18]:
data_copy.head()

,Tower Name,Application Name,Email Subject,Flag,combined
0,Core Pharmacy,PharmRDS-Accredo RealTime Messages,Production - Accredo Real Time Order Process S...,Actionable,Core Pharmacy PharmRDS-Accredo RealTime Messag...
1,Core Pharmacy,IVR - Web Service,Splunk Alert: SO IVR - Pharmacy Migration Coor...,Actionable,Core Pharmacy IVR - Web Service Splunk Alert: ...
2,Core Pharmacy,ISAM- Web Service,Splunk Alert: ISAM - Slowness,Actionable,Core Pharmacy ISAM- Web Service Splunk Alert: ...
3,Core Pharmacy,Migration Co-Ordinaor,Splunk Alert: Migration_Coordinator_500 Clone,Actionable,Core Pharmacy Migration Co-Ordinaor Splunk Ale...
4,Core Pharmacy,Claim Realm - Web Service,Splunk Alert: Claim Realm Service - Slowness A...,Actionable,Core Pharmacy Claim Realm - Web Service Splunk...


### Preprocessing

In [49]:
# def Preprocessing_stemming(df,column_name):
#     # Email Subject column
#     df[column_name] = df[column_name].apply(lambda x: re.sub('[^A-Za-z]'," ",x))
#     df[column_name] = df[column_name].replace([" ","  ","   "]," ",regex = True)
#     stopwords = set(STOPWORDS) 
#     stemmer = PorterStemmer()
#     df[column_name] = df[column_name].apply(lambda x : x.lower())
#     df[column_name] = df[column_name].apply(lambda x : " ".join(word for word in x if len(word) > 2))
#     df[column_name] = df[column_name].apply(lambda x : " ".join([word for word in x.split() if word not in stopwords]))
#     df[column_name] = df[column_name].apply(lambda x : stemmer.stem(x)) 
#     return df[column_name]


# def Preprocessing_lemmatization(df,column_name):
#     # Email Subject column
#     df[column_name] = df[column_name].apply(lambda x: re.sub('[^A-Za-z]'," ",x))
#     df[column_name] = df[column_name].replace([" ","  ","   "]," ",regex = True)
#     stopwords = set(STOPWORDS) 
#     lemmatizer = WordNetLemmatizer()
#     df[column_name] = df[column_name].apply(lambda x : x.lower())
#     df[column_name] = df[column_name].apply(lambda x : " ".join(word for word in x if len(word) > 2))
#     df[column_name] = df[column_name].apply(lambda x : " ".join([word for word in x.split() if word not in stopwords]))
#     df[column_name] = df[column_name].apply(lambda x : lemmatizer.lemmatize(x)) 
#     return df[column_name]


def Preprocessing_lemmatization_spacy(df,column_name):
    # Email Subject column
    df[column_name] = df[column_name].apply(lambda x: re.sub('[^A-Za-z]'," ",x))
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    df[column_name] = df[column_name].apply(lambda x: _RE_COMBINE_WHITESPACE.sub(" ",x).strip())
    stopwords = set(STOPWORDS) 
    nlp = spacy.load("en_core_web_sm")
    df[column_name] = df[column_name].apply(lambda x : x.lower())
    df[column_name] = df[column_name].apply(lambda x : " ".join(word for word in x.split() if len(word) > 2))
    df[column_name] = df[column_name].apply(lambda x : " ".join([word for word in x.split() if word not in stopwords]))
    df[column_name] = df[column_name].apply(lambda x : " ".join(token.lemma_ for token in nlp(x))) 
    return df[column_name]


In [50]:
data_copy["combined"] = Preprocessing_lemmatization_spacy(data_copy, "combined")
data_copy.head()

,Tower Name,Application Name,Email Subject,Flag,combined
0,Core Pharmacy,PharmRDS-Accredo RealTime Messages,Production - Accredo Real Time Order Process S...,Actionable,core pharmacy pharmrd accredo realtime message...
1,Core Pharmacy,IVR - Web Service,Splunk Alert: SO IVR - Pharmacy Migration Coor...,Actionable,core pharmacy ivr web service splunk alert ivr...
2,Core Pharmacy,ISAM- Web Service,Splunk Alert: ISAM - Slowness,Actionable,core pharmacy isam web service splunk alert is...
3,Core Pharmacy,Migration Co-Ordinaor,Splunk Alert: Migration_Coordinator_500 Clone,Actionable,core pharmacy migration ordinaor splunk alert ...
4,Core Pharmacy,Claim Realm - Web Service,Splunk Alert: Claim Realm Service - Slowness A...,Actionable,core pharmacy claim realm web service splunk a...


In [51]:
X = data_copy["combined"]
y = data_copy["Flag"].map({"Actionable" : 0, "Non Actionable" : 1})

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

In [52]:
from sklearn.pipeline import Pipeline

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import gensim
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf_clf', RandomForestClassifier()),
])

params = {"tfidf__ngram_range" : [(1,2),(1,1)],
         "tfidf__max_df" : [4,5,6,7,8],
         "tfidf__min_df" : [1,2,3],
         "tfidf__max_features" : [50,100,150,200],
         "rf_clf__n_estimators" : [100,200,400],
         "rf_clf__max_depth": [2,3,4],
         "rf_clf__min_samples_split" : [2,3,4,5]}

from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline,param_grid= params, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(params)
grid_search.fit(X_train,y_train)

AttributeError: module 'smart_open' has no attribute 'local_file'

In [ ]:
grid_search.best_estimator_.steps

In [23]:
best_rf = grid_search.best_estimator_
best_rf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=8, max_features=150, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [26]:
## prediction
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

prediction_rf_train = best_rf.predict(X_train)

print("---------------Training Result random forest--------------- \n \n")
print(confusion_matrix(y_train,prediction_rf_train))
print("accuracy -- " , accuracy_score(y_train,prediction_rf_train))
print("precision_score -- " , precision_score(y_train,prediction_rf_train))
print("recall_score -- " , recall_score(y_train,prediction_rf_train))
print("f1_score -- " , f1_score(y_train,prediction_rf_train))
print("classification report --- \n",classification_report(y_train,prediction_rf_train))

prediction_rf_test = best_rf.predict(X_test)

print("---------------Testing Result random forest--------------- \n \n")
print(confusion_matrix(y_test,prediction_rf_test))
print("accuracy -- " , accuracy_score(y_test,prediction_rf_test))
print("precision_score -- " , precision_score(y_test,prediction_rf_test))
print("recall_score -- " , recall_score(y_test,prediction_rf_test))
print("f1_score -- " , f1_score(y_test,prediction_rf_test))
print("classification report --- \n",classification_report(y_test,prediction_rf_test))

---------------Training Result Xgboost--------------- 
 

[[26  1]
 [ 8  4]]
accuracy --  0.7692307692307693
precision_score --  0.8
recall_score --  0.3333333333333333
f1_score --  0.47058823529411764
classification report --- 
               precision    recall  f1-score   support

           0       0.76      0.96      0.85        27
           1       0.80      0.33      0.47        12

   micro avg       0.77      0.77      0.77        39
   macro avg       0.78      0.65      0.66        39
weighted avg       0.78      0.77      0.73        39

---------------Testing Result Xgboost--------------- 
 

[[18  3]
 [ 6  0]]
accuracy --  0.6666666666666666
precision_score --  0.0
recall_score --  0.0
f1_score --  0.0
classification report --- 
               precision    recall  f1-score   support

           0       0.75      0.86      0.80        21
           1       0.00      0.00      0.00         6

   micro avg       0.67      0.67      0.67        27
   macro avg       0.38    

#### TFIDF with RF not using pipeline

##### First

In [54]:
vectorizer = TfidfVectorizer(use_idf = True, ngram_range = (1,1),max_df = 15,min_df = 1,max_features = 100,norm= 'l2')
train_data = vectorizer.fit_transform(X_train)
test_data = vectorizer.transform(X_test)
train_data.shape

(39, 100)

In [55]:
import pickle
# Dump the file
pickle.dump(vectorizer, open("tfidf.pkl", "wb"))

In [56]:
vectorizer.vocabulary_

{'dental': 33,
 'warning': 93,
 'abort': 1,
 'attn': 8,
 'home': 41,
 'delivery': 32,
 'team': 86,
 'fairfax': 39,
 'splunk': 81,
 'alert': 3,
 'cache': 15,
 'file': 40,
 'exception': 37,
 'blockers': 13,
 'webprd': 96,
 'pdm': 57,
 'subject': 84,
 'contain': 28,
 'pricing': 64,
 'prd': 63,
 'xrx': 99,
 'application': 7,
 'open': 53,
 'problem': 65,
 'failure': 38,
 'rate': 69,
 'increase': 42,
 'web': 95,
 'service': 77,
 'chc': 19,
 'prod': 67,
 'chdpxrxtransactionservicewshpv': 22,
 'log': 48,
 'update': 91,
 'last': 45,
 'chdpxrxtransacionservicewshttpv': 21,
 'mailrx': 49,
 'router': 75,
 'miss': 50,
 'pdf': 56,
 'report': 72,
 'system': 85,
 'unavailable': 90,
 'wdc': 94,
 'dna': 34,
 'rds': 70,
 'word': 98,
 'ohip': 52,
 'welcome': 97,
 'clinicla': 26,
 'cmg': 27,
 'allscripts': 5,
 'critical': 30,
 'ciwinp': 24,
 'lgenl': 46,
 'drive': 35,
 'space': 80,
 'available': 10,
 'apm': 6,
 'chdp': 20,
 'qdepth': 68,
 'state': 83,
 'request': 73,
 'core': 29,
 'pharmacy': 59,
 'ivr': 4

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth = 3,
                                 max_features = None,
                                 n_estimators = 150,
                                 min_samples_split = 3,
                                 min_samples_leaf = 2,
                                 random_state = 10)
rf_model.fit(train_data,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [62]:
## prediction
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

prediction_rf_train = rf_model.predict(train_data)

print("---------------Training Result random forest--------------- \n \n")
print(confusion_matrix(y_train,prediction_rf_train))
print("accuracy -- " , accuracy_score(y_train,prediction_rf_train))
print("precision_score -- " , precision_score(y_train,prediction_rf_train))
print("recall_score -- " , recall_score(y_train,prediction_rf_train))
print("f1_score -- " , f1_score(y_train,prediction_rf_train))
print("classification report --- \n",classification_report(y_train,prediction_rf_train))

prediction_rf_test = rf_model.predict(test_data)

print("---------------Testing Result random forest--------------- \n \n")
print(confusion_matrix(y_test,prediction_rf_test))
print("accuracy -- " , accuracy_score(y_test,prediction_rf_test))
print("precision_score -- " , precision_score(y_test,prediction_rf_test))
print("recall_score -- " , recall_score(y_test,prediction_rf_test))
print("f1_score -- " , f1_score(y_test,prediction_rf_test))
print("classification report --- \n",classification_report(y_test,prediction_rf_test))

---------------Training Result random forest--------------- 
 

[[25  2]
 [ 0 12]]
accuracy --  0.9487179487179487
precision_score --  0.8571428571428571
recall_score --  1.0
f1_score --  0.923076923076923
classification report --- 
               precision    recall  f1-score   support

           0       1.00      0.93      0.96        27
           1       0.86      1.00      0.92        12

   micro avg       0.95      0.95      0.95        39
   macro avg       0.93      0.96      0.94        39
weighted avg       0.96      0.95      0.95        39

---------------Testing Result random forest--------------- 
 

[[20  1]
 [ 1  5]]
accuracy --  0.9259259259259259
precision_score --  0.8333333333333334
recall_score --  0.8333333333333334
f1_score --  0.8333333333333334
classification report --- 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        21
           1       0.83      0.83      0.83         6

   micro avg       0.93   

Notes :- 
    - max_df = (13 - 18)
    - min_df = (1 - 3)
    - max_depth = (2 - 3)
    - n_estimators = (100 - 200)

In [242]:
import pickle

In [246]:
with open('random_forest_tfidf.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

##### Second

In [251]:
def is_phrase_in(phrase, text):
        return re.search(r"\b{}\b".format(phrase), text, re.IGNORECASE) is not None


def replace_bigram(sentence,dict_):
    for bigram in dict_.keys():
        if is_phrase_in(bigram,sentence):
            sentence = sentence.replace(bigram,dict_[bigram])
    return sentence

In [303]:

doc = " ".join(sent for sent in X_train)
wordcloud = WordCloud(width = 800, height = 800, 
            background_color ='white', 
            stopwords = stopwords, 
            min_font_size = 20,
            collocation_threshold = 18).generate(doc) 

top_50_words = list(wordcloud.words_.keys())[:90]
# changing bigrams to unigrams in X_train for word_embedding

change_dict = {}
for word in top_50_words:
    if len(word.split()) > 1:
        change_dict[word] = word.replace(" ","_")

top_50_words_changed = [change_dict[word] if word in list(change_dict.keys()) else word for word in top_50_words] 


X_train_copy = X_train.copy()
X_train_copy = X_train_copy.apply(lambda x: replace_bigram(x,change_dict))

# create tfidf based on these top_50_words
tfidf_1 = TfidfVectorizer(vocabulary = top_50_words_changed)
new_train_data = tfidf_1.fit_transform(X_train_copy)

X_test_copy = X_test.copy()
X_test_copy = X_test_copy.apply(lambda x: replace_bigram(x,change_dict))
new_test_data = tfidf_1.transform(X_test_copy)

In [304]:
rf_model = RandomForestClassifier(max_depth = 3,
                                 max_features = None,
                                 n_estimators = 150,
                                 min_samples_split = 3,
                                 min_samples_leaf = 2,
                                 random_state = 10)
rf_model.fit(new_train_data,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [305]:
## prediction
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

prediction_rf_train1 = rf_model.predict(new_train_data)

print("---------------Training Result random forest--------------- \n \n")
print(confusion_matrix(y_train,prediction_rf_train1))
print("accuracy -- " , accuracy_score(y_train,prediction_rf_train1))
print("precision_score -- " , precision_score(y_train,prediction_rf_train1))
print("recall_score -- " , recall_score(y_train,prediction_rf_train1))
print("f1_score -- " , f1_score(y_train,prediction_rf_train1))
print("classification report --- \n",classification_report(y_train,prediction_rf_train1))

prediction_rf_test1 = rf_model.predict(new_test_data)

print("---------------Testing Result random forest--------------- \n \n")
print(confusion_matrix(y_test,prediction_rf_test1))
print("accuracy -- " , accuracy_score(y_test,prediction_rf_test1))
print("precision_score -- " , precision_score(y_test,prediction_rf_test1))
print("recall_score -- " , recall_score(y_test,prediction_rf_test1))
print("f1_score -- " , f1_score(y_test,prediction_rf_test1))
print("classification report --- \n",classification_report(y_test,prediction_rf_test1))

---------------Training Result random forest--------------- 
 

[[25  2]
 [ 1 11]]
accuracy --  0.9230769230769231
precision_score --  0.8461538461538461
recall_score --  0.9166666666666666
f1_score --  0.8799999999999999
classification report --- 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94        27
           1       0.85      0.92      0.88        12

   micro avg       0.92      0.92      0.92        39
   macro avg       0.90      0.92      0.91        39
weighted avg       0.93      0.92      0.92        39

---------------Testing Result random forest--------------- 
 

[[20  1]
 [ 2  4]]
accuracy --  0.8888888888888888
precision_score --  0.8
recall_score --  0.6666666666666666
f1_score --  0.7272727272727272
classification report --- 
               precision    recall  f1-score   support

           0       0.91      0.95      0.93        21
           1       0.80      0.67      0.73         6

   micro avg       0.89  

##### Third

In [25]:
data_copy["Email Subject"] = Preprocessing_lemmatization_spacy(data_copy, "Email Subject")
X1 = data_copy["Email Subject"]
y1 = data_copy["Flag"].map({"Actionable" : 0, "Non Actionable" : 1})

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.40, random_state=42)
training_index = X_train1.index
testing_index = X_test1.index

In [26]:
training_index

Int64Index([66, 65, 57, 31, 51, 71, 15, 27, 26, 24, 54, 11, 32, 68, 53, 37, 29,
            50, 56,  1, 21,  2, 60, 39, 35, 59, 23, 62, 10, 22, 18, 64, 38, 20,
            67,  7, 49, 14, 58],
           dtype='int64')

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import gensim
from sklearn.ensemble import RandomForestClassifier

vectorizer = TfidfVectorizer(use_idf = True, ngram_range = (1,2),max_df = 10,min_df = 3,max_features = 200,norm= 'l2')
train_data = vectorizer.fit_transform(X_train1)
test_data = vectorizer.transform(X_test1)
train_data.shape

(39, 29)

In [116]:
vectorizer.vocabulary_

{'warning': 24,
 'splunk': 20,
 'alert': 0,
 'file': 6,
 'splunk alert': 21,
 'subject': 22,
 'contain': 3,
 'subject contain': 23,
 'open': 11,
 'problem': 13,
 'failure': 4,
 'rate': 16,
 'increase': 7,
 'web': 25,
 'service': 18,
 'chc': 1,
 'xrx': 27,
 'prod': 15,
 'open problem': 12,
 'problem failure': 14,
 'failure rate': 5,
 'rate increase': 17,
 'increase web': 8,
 'web service': 26,
 'service chc': 19,
 'chc xrx': 2,
 'xrx prod': 28,
 'log': 10,
 'last': 9}

In [117]:
# Build dataframe
train_data = pd.DataFrame(train_data.toarray(),columns =vectorizer.vocabulary_,index = training_index)
test_data = pd.DataFrame(test_data.toarray(),columns =vectorizer.vocabulary_,index = testing_index)

In [118]:
train_data.head()

,warning,splunk,alert,file,splunk alert,subject,contain,subject contain,open,problem,...,problem failure,failure rate,rate increase,increase web,web service,service chc,chc xrx,xrx prod,log,last
66,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
65,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.430275,0.0,0.0,0.0,0.0,0.0,0.578357,0.0,0.0,0.0,...,0.0,0.490085,0.490085,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
# dummy other columns in data_copy
dummies = pd.get_dummies(data_copy[["Tower Name","Application Name"]],drop_first= True)
dummies

,Tower Name_Core Pharmacy,Tower Name_Coresol,Tower Name_DNA,Tower Name_Dental,Tower Name_Home Delivery Team,Application Name_AllScripts,Application Name_Cerner,Application Name_Claim Realm - Web Service,Application Name_Claims RDS,Application Name_DrugPricing - Web Service,...,Application Name_ODS,Application Name_PDA,Application Name_PDM,Application Name_PRDS,Application Name_PharmRDS-Accredo RealTime Messages,Application Name_PharmRDS-IIB Alert,Application Name_PharmRDS-NRT Alert,Application Name_SRWSQLXPW001.internal.cigna.com,Application Name_UC4,Application Name_xRx Application
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
69,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
71,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [120]:
training_index

Int64Index([66, 65, 57, 31, 51, 71, 15, 27, 26, 24, 54, 11, 32, 68, 53, 37, 29,
            50, 56,  1, 21,  2, 60, 39, 35, 59, 23, 62, 10, 22, 18, 64, 38, 20,
            67,  7, 49, 14, 58],
           dtype='int64')

In [121]:
training_dummies = dummies.loc[training_index,:]
testing_dummies = dummies.loc[testing_index,:]

In [122]:
train_data = pd.concat([train_data,training_dummies],axis = 1)
test_data = pd.concat([test_data,testing_dummies],axis = 1)

In [135]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth = 3,
                                 max_features = None,
                                 n_estimators = 600,
                                 min_samples_split = 3,
                                 min_samples_leaf = 2,
                                 random_state = 10)
rf_model.fit(train_data,y_train1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [136]:
## prediction
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

prediction_rf_train = rf_model.predict(train_data)

print("---------------Training Result random forest--------------- \n \n")
print(confusion_matrix(y_train1,prediction_rf_train))
print("accuracy -- " , accuracy_score(y_train1,prediction_rf_train))
print("precision_score -- " , precision_score(y_train1,prediction_rf_train))
print("recall_score -- " , recall_score(y_train1,prediction_rf_train))
print("f1_score -- " , f1_score(y_train1,prediction_rf_train))
print("classification report --- \n",classification_report(y_train1,prediction_rf_train))

prediction_rf_test = rf_model.predict(test_data)

print("---------------Testing Result random forest--------------- \n \n")
print(confusion_matrix(y_test1,prediction_rf_test))
print("accuracy -- " , accuracy_score(y_test1,prediction_rf_test))
print("precision_score -- " , precision_score(y_test1,prediction_rf_test))
print("recall_score -- " , recall_score(y_test1,prediction_rf_test))
print("f1_score -- " , f1_score(y_test1,prediction_rf_test))
print("classification report --- \n",classification_report(y_test1,prediction_rf_test))

---------------Training Result random forest--------------- 
 

[[22  5]
 [ 0 12]]
accuracy --  0.8717948717948718
precision_score --  0.7058823529411765
recall_score --  1.0
f1_score --  0.8275862068965517
classification report --- 
               precision    recall  f1-score   support

           0       1.00      0.81      0.90        27
           1       0.71      1.00      0.83        12

   micro avg       0.87      0.87      0.87        39
   macro avg       0.85      0.91      0.86        39
weighted avg       0.91      0.87      0.88        39

---------------Testing Result random forest--------------- 
 

[[19  2]
 [ 1  5]]
accuracy --  0.8888888888888888
precision_score --  0.7142857142857143
recall_score --  0.8333333333333334
f1_score --  0.7692307692307692
classification report --- 
               precision    recall  f1-score   support

           0       0.95      0.90      0.93        21
           1       0.71      0.83      0.77         6

   micro avg       0.89  

Here our aim will be any actionable maill shoul not be classified as non actionable.so we have to decrease false positive for non actionable as positive. which means we have to reduce precision for non actionable as positive ((1,2) cell in confusion matrix).

In [134]:
# SVM
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(max_iter = 400,
                   C = 1,
                   class_weight = 'balanced')


# params = {'max_iter': (200,400,500,1000),
#          "C" : (.001,.002,.005,.009,.01,.02,.05,1),
#          "class_weight" : [None,'balanced']}

# from sklearn.model_selection import GridSearchCV
# grid_search = GridSearchCV(svm_clf,param_grid= params, n_jobs=-1, verbose=1)


# grid_search.fit(train_data,y_tr)

# best_svm_clf = grid_search.best_estimator_
# print("best svm features -- \n",best_svm_clf)
# print("\n")

svm_clf.fit(train_data,y_train1)

## prediction

prediction_best_svm_train = svm_clf.predict(train_data)

print("---------------Training Result SVM--------------- \n \n")
print(confusion_matrix(y_train1,prediction_best_svm_train))
print("accuracy -- " , accuracy_score(y_train1,prediction_best_svm_train))
print("precision_score -- " , precision_score(y_train1,prediction_best_svm_train))
print("recall_score -- " , recall_score(y_train1,prediction_best_svm_train))
print("f1_score -- " , f1_score(y_train1,prediction_best_svm_train))
print("classification report --- \n",classification_report(y_train1,prediction_best_svm_train))

prediction_best_svm_test = svm_clf.predict(test_data)

print("classification report --- \n",classification_report(y_test1,prediction_best_svm_test))
print("---------------Testing Result SVM--------------- \n \n")
print(confusion_matrix(y_test1,prediction_best_svm_test))
print("accuracy -- " , accuracy_score(y_test1,prediction_best_svm_test))
print("precision_score -- " , precision_score(y_test1,prediction_best_svm_test))
print("recall_score -- " , recall_score(y_test1,prediction_best_svm_test))
print("f1_score -- " , f1_score(y_test1,prediction_best_svm_test))

---------------Training Result SVM--------------- 
 

[[22  5]
 [ 0 12]]
accuracy --  0.8717948717948718
precision_score --  0.7058823529411765
recall_score --  1.0
f1_score --  0.8275862068965517
classification report --- 
               precision    recall  f1-score   support

           0       1.00      0.81      0.90        27
           1       0.71      1.00      0.83        12

   micro avg       0.87      0.87      0.87        39
   macro avg       0.85      0.91      0.86        39
weighted avg       0.91      0.87      0.88        39

classification report --- 
               precision    recall  f1-score   support

           0       0.95      0.90      0.93        21
           1       0.71      0.83      0.77         6

   micro avg       0.89      0.89      0.89        27
   macro avg       0.83      0.87      0.85        27
weighted avg       0.90      0.89      0.89        27

---------------Testing Result SVM--------------- 
 

[[19  2]
 [ 1  5]]
accuracy --  0.888888

#### Word2Vec with RF not using pipeline¶

In [247]:
def word_embedding_feature(tokenized_data):
    wordvec_arrays = np.zeros((len(tokenized_data), 50)) 
    
    for i in range(len((tokenized_data))):
        size = len(tokenized_data[i])
        aggregated_wv = np.zeros(50)

        for word in tokenized_data[i]:
            try:
                aggregated_wv += model_w2v[word]
            except:
                aggregated_wv += np.zeros(50)

        aggregated_wv = aggregated_wv / size
        wordvec_arrays[i] = aggregated_wv
    return pd.DataFrame(wordvec_arrays) 


def feature_extraction_2(df_train,df_test = None):
    train_tokenized_mails = df_train.apply(lambda x : x.split()).reset_index(drop= 'first')
    model_w2v = gensim.models.Word2Vec(train_tokenized_mails,
                          size = 50,
                          window = 3,
                          min_count = 1,
                          sg = 1,
                          negative = 5,
                          workers = 4,
                          seed = 34)

    model_w2v.train(train_tokenized_mails, total_examples= len(df_train), epochs=20)
    
    print("Features -- \n", model_w2v.wv.vocab.keys())
    train_wordvec_df = word_embedding_feature(train_tokenized_mails)

    if df_test != None:
        test_tokenized_mails = df_test.apply(lambda x : x.split()).reset_index(drop= 'first')
        test_wordvec_df = word_embedding_feature(test_tokenized_mails)
    else:
        test_wordvec_df = None
    return train_wordvec_df,test_wordvec_df

## Testing

In [84]:
import pandas as pd
import numpy as np
import re
from wordcloud import STOPWORDS 
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer 

def prediction_single_observation(tower_name,application_name,email_subject,email_body):
    
    # input
    tower_nm = str(tower_name)
    app_nm = str(application_name)
    email_sb = str(email_subject)
    email_bdy = ""
    
    #preprocessing
    combined = tower_nm + " " + app_nm + " " + email_sb + " " + email_bdy
    
    combined = re.sub('[^A-Za-z]'," ",combined)
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    combined = _RE_COMBINE_WHITESPACE.sub(" ", combined).strip()
    stopwords = set(STOPWORDS) 
    nlp = spacy.load("en_core_web_sm")
    combined = combined.lower()
    combined = " ".join(word for word in combined.split() if len(word) > 2)
    combined = " ".join([word for word in combined.split() if word not in stopwords])
    combined = " ".join(token.lemma_ for token in nlp(combined))
    
    # Feature Extraction
    # load the feature extarctor
    tf_idf = pickle.load(open("tfidf.pkl", 'rb'))
    
    combined = pd.Series(combined)
    test_data = tf_idf.transform(combined)
    
    # load the model from disk
    loaded_model = pickle.load(open("random_forest_tfidf.pkl", 'rb'))
    result = loaded_model.predict(test_data)
    
    if result[0] == 0:
        print("Actionable")
        
    else:
        print("Non Actionable")

        
        
    

def prediction_data(path):
    
    # Reading
    try:
        data = pd.read_excel(path) 
    except:
        data = pd.read_csv(path) 
    
    
    #preprocessing
    data['combined'] = data.apply(lambda x : f"{x[0]} {x[1]} {x[2]}",axis = 1)
    data["combined"] = Preprocessing_lemmatization_spacy(data, "combined")
    
    
    # Feature Extraction
    # load the feature extarctor
    tf_idf = pickle.load(open("tfidf.pkl", 'rb'))
    test_data = tf_idf.transform(data["combined"])
    
    # load the model 
    loaded_model = pickle.load(open("random_forest_tfidf.pkl", 'rb'))
    result = loaded_model.predict(test_data)
    #print(result)
    
    #concat prediction
    data["Flag"] = result
    data["Flag"] = data["Flag"].map({0 : "Actionable" , 1 : "Non Actionable"})
    
    data.to_csv("result_data.csv")
    print("check data in ypur current directory")

In [80]:
prediction_single_observation('Core Pharmacy' ,' PharmRDS-Accredo RealTime Messages','error alert please solve',"")

Actionable


In [85]:
prediction_data("Book1.xlsx")

check data in ypur current directory


In [ ]:
!conda install -c conda-forge pywin32

In [2]:
!py -3.7 -m pip install pywin32

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!python C:\Users\Chayan\Anaconda3\Scripts\pywin32_postinstall.py -install

Parsed arguments are: Namespace(destination='C:\\Users\\Chayan\\Anaconda3\\Lib\\site-packages', install=True, quiet=False, remove=False, silent=False, wait=None)

Copied pythoncom37.dll to C:\Users\Chayan\Anaconda3\pythoncom37.dll

Copied pywintypes37.dll to C:\Users\Chayan\Anaconda3\pywintypes37.dll

You do not have the permissions to install COM objects.

The sample COM objects were not registered.

-> Software\Python\PythonCore\3.7\Help[None]=None

-> Software\Python\PythonCore\3.7\Help\Pythonwin Reference[None]='C:\\Users\\Chayan\\Anaconda3\\Lib\\site-packages\\PyWin32.chm'

Pythonwin has been registered in context menu

Shortcut for Pythonwin created

Shortcut to documentation created

The pywin32 extensions were successfully installed.



C:\Users\Chayan\Anaconda3\Scripts\pywin32_postinstall.py:164: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses

  import imp



In [5]:
import os
print(os.environ["PATH"])

C:\Users\Chayan\Anaconda3\lib\site-packages\pywin32_system32;C:\Users\Chayan\Anaconda3\lib\site-packages\pywin32_system32;C:\Users\Chayan\Anaconda3\lib\site-packages\pywin32_system32;C:\Users\Chayan\Anaconda3;C:\Users\Chayan\Anaconda3\Library\mingw-w64\bin;C:\Users\Chayan\Anaconda3\Library\usr\bin;C:\Users\Chayan\Anaconda3\Library\bin;C:\Users\Chayan\Anaconda3\Scripts;C:\Users\Chayan\Anaconda3\bin;C:\Users\Chayan\Anaconda3\condabin;C:\Users\Chayan\Anaconda3\lib\site-packages\pywin32_system32;C:\Users\Chayan\Anaconda3\condabin\..;C:\Users\Chayan\Anaconda3\condabin\..\Library\mingw-w64\bin;C:\Users\Chayan\Anaconda3\condabin\..\Library\usr\bin;C:\Users\Chayan\Anaconda3\condabin\..\Library\bin;C:\Users\Chayan\Anaconda3\condabin\..\Scripts;C:\Users\Chayan\Anaconda3\condabin\..\bin;C:\Program Files\Java\jdk1.8.0_211\bin;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Users\Chayan\A

In [9]:
import win32com.client

NameError: name 'getwd' is not defined

In [ ]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

inbox = outlook.GetDefaultFolder(6) # "6" refers to the index of a folder - in this case,
                                    # the inbox. You can change that number to reference
                                    # any other folder
messages = inbox.Items
message = messages.GetLast()
body_content = message.body
print(body_content)